<h1 div class='alert alert-success'><center> Ponto de partida (EDA, linha de base)</center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# Descrição de dados

Para esta competição, você vai prever se um cliente fez uma reclamação sobre uma apólice de seguro. A verdade fundamental claimtem valor binário, mas uma previsão pode ser qualquer número de 0.0 para 1.0, representando a probabilidade de uma reclamação. Os recursos neste conjunto de dados foram tornados anônimos e podem conter valores ausentes.
arquivos

- `train.csv`: os dados de treinamento com o alvo claimcoluna
- `test.csv`: o conjunto de teste; você estará prevendo o claimpara cada linha neste arquivo
- `sample_submission.csv`:  um arquivo de envio de amostra no formato correto

# <div class="alert alert-success">  1. IMPORTAÇÕES </div> 

## 1.1. Bibliotecas 

In [ ]:
import warnings
import random
import os
import gc
import torch

In [ ]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt 
import seaborn           as sns
import joblib            as jb
import scikitplot        as skplt

In [ ]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing   import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
from sklearn.impute          import SimpleImputer
from sklearn                 import metrics

In [ ]:
import xgboost               as xgb

## 1.2. Funções
Aqui centralizamos todas as funções desenvolvidas durante o projeto para melhor organização do código.

In [ ]:
def jupyter_setting():
    
    %matplotlib inline
      
    #os.environ["WANDB_SILENT"] = "true" 
    #plt.style.use('bmh') 
    #plt.rcParams['figure.figsize'] = [20,15]
    #plt.rcParams['font.size']      = 13
     
    pd.options.display.max_columns = None
    #pd.set_option('display.expand_frame_repr', False)

    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    #warnings.filterwarnings(category=UserWarning)

    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

    pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

# Colors
dark_red = "#b20710"
black    = "#221f1f"
green    = "#009473"
myred    = '#CD5C5C'
myblue   = '#6495ED'
mygreen  = '#90EE90'

cols= [myred, myblue,mygreen]

In [ ]:
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
def missing_zero_values_table(df):
        mis_val         = df.isnull().sum()
        mis_val_percent = round(df.isnull().mean().mul(100), 2)
        mz_table        = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table        = mz_table.rename(columns = {df.index.name:'col_name', 
                                                     0 : 'Valores ausentes', 
                                                     1 : '% de valores totais'})
        
        mz_table['Tipo de dados'] = df.dtypes
        mz_table                  = mz_table[mz_table.iloc[:,1] != 0 ]. \
                                     sort_values('% de valores totais', ascending=False)
        
        msg = "Seu dataframe selecionado tem {} colunas e {} " + \
              "linhas. \nExistem {} colunas com valores ausentes."
            
        print (msg.format(df.shape[1], df.shape[0], mz_table.shape[0]))
        
        return mz_table.reset_index()

In [ ]:
def describe(df):
    var = df.columns

    # Medidas de tendência central, média e mediana 
    ct1 = pd.DataFrame(df[var].apply(np.mean)).T
    ct2 = pd.DataFrame(df[var].apply(np.median)).T

    # Dispensão - str, min , max range skew, kurtosis
    d1 = pd.DataFrame(df[var].apply(np.std)).T
    d2 = pd.DataFrame(df[var].apply(min)).T
    d3 = pd.DataFrame(df[var].apply(max)).T
    d4 = pd.DataFrame(df[var].apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df[var].apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df[var].apply(lambda x: x.kurtosis())).T
    d7 = pd.DataFrame(df[var].apply(lambda x: (3 *( np.mean(x) - np.median(x)) / np.std(x) ))).T

    # concatenete 
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6, d7]).T.reset_index()
    m.columns = ['attrobutes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis','coef_as']
    
    return m

In [ ]:
def reduce_memory_usage(df, verbose=True):
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
        
    return df

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [ ]:
def confusion_plot(matrix, labels = None, title = None):
        
    labels = labels if labels else ['Negative (0)', 'Positive (1)']    
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    
    sns.heatmap(data        = matrix, 
                cmap        = 'Blues', 
                annot       = True, 
                fmt         = 'd',
                xticklabels = labels, 
                yticklabels = labels, 
                ax          = ax);
    
    ax.set_xlabel('\n PREVISTO', fontsize=15)
    ax.set_ylabel('REAL \n', fontsize=15)
    ax.set_title(title)
    
    plt.close();
    
    return fig;

In [ ]:
!mkdir img
!mkdir Data
!mkdir Data/pkl
!mkdir Data/submission

!mkdir model
!mkdir model/preds
!mkdir model/optuna

!mkdir model/preds/test
!mkdir model/preds/test/n1
!mkdir model/preds/test/n2
!mkdir model/preds/test/n3

!mkdir model/preds/train
!mkdir model/preds/train/n1
!mkdir model/preds/train/n2
!mkdir model/preds/train/n3
!mkdir model/preds/param

## 1.4. Carregar Dados
São dois arquivos que vamos utilizar para análise e treinanmento dos modelos, e um arquivo para submissão na competição.

- `train.csv`: arquivo com dados de treinamento;  
- `test.csv`: arquivo que será utilizado para previsão; 
- `sample_submission.csv`: arquivo utlizado para envio das previsões.  


In [ ]:
path = '../input/tabular-playground-series-nov-2021/'

In [ ]:
%%time

df1_train     = pd.read_csv(path + 'train.csv')
df1_test      = pd.read_csv(path + 'test.csv')
df_submission = pd.read_csv(path + 'sample_submission.csv')

df1_train.shape, df1_test.shape, df_submission.shape

In [ ]:
df1_train.head()

In [ ]:
df1_test.head()

# <div class="alert alert-success">  1.0. Análise Exploratória de Dados (EDA)  </div> 

### 1.1.2. Dimensão do DataSet

In [ ]:
print('TREINO')
print('Number of Rows: {}'.format(df1_train.shape[0]))
print('Number of Columns: {}'.format(df1_train.shape[1]), end='\n\n')

print('TESTE')
print('Number of Rows: {}'.format(df1_test.shape[0]))
print('Number of Columns: {}'.format(df1_test.shape[1]))

### 1.1.3. Tipo de Dados

In [ ]:
df1_train.info()

In [ ]:
df1_test.info()

In [ ]:
print(f'{3*"="} For Pandas {10*"="}\n{(df1_train.dtypes).value_counts()}')
print(f'\n{3*"="} For Datatable {7*"="}\n{(df1_test.dtypes).value_counts()}')

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <BR>
    
- O dataset de treiro tem 466.9 MB com 600000 de registros e 102 columas; 
- O dataset de teste tem 416.1 com 540000 de registros e 101 columas
   
    
Vamos fazer uma redução desses dataset nas próximas etapas, primeiro vamos identificar os tipos de dados que temos nos datasets.

</div>

### 1.1.4. Idenficar Variáveis Ausentes (NA)
Vamos verificar os valores ausentes em cada variável conjunto de treinono e teste.

In [ ]:
missing = missing_zero_values_table(df1_train)
missing[:].style.background_gradient(cmap='Reds')

In [ ]:
missing = missing_zero_values_table(df1_test)
missing[:].style.background_gradient(cmap='Reds')

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>

Não temos dados faltantes.
    
</div>

### 1.1.6. Estatística Descritiva
Abaixo estão as estatísticas básicas para cada variável que contém informações sobre contagem, média, desvio padrão, mínimo, 1º quartil, mediana, 3º quartil e máximo.

In [ ]:
feature_cat   = df1_test.select_dtypes(object).columns.to_list()
feature_float = df1_test.select_dtypes(np.number).columns.to_list()

In [ ]:
print('Temos {} variávies numéricas e {} categóricas.'.format(len(feature_float), len(feature_cat)))

#### 1.1.6.1. Atributos Numéricos

- Train

In [ ]:
df1_train[feature_float].describe().style.background_gradient(cmap='YlOrRd')

- Test

In [ ]:
df1_test[feature_float].describe().style.background_gradient(cmap='YlOrRd')

#### 1.1.6.1. Atributos Categóricos

In [ ]:
df1_train[feature_cat].columns

## 1.2. Análise Gráfica

### 1.2.1. Correlação
Vamos examinar a correlação entre as variáveis.

In [ ]:
df = df1_train[feature_float].corr().round(5)

# Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(16,16))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

plt.setp(ax.get_xticklabels(), 
         rotation      = 90, 
         ha            = "right",
         rotation_mode = "anchor", 
         weight        = "normal")

plt.setp(ax.get_yticklabels(), 
         weight        = "normal",
         rotation_mode = "anchor", 
         rotation      = 0, 
         ha            = "right");

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Como podemos observar, a correlação está entre -0.075 e 0.1, o que é muito pequeno, portanto, as variáveis são fracamente correlacionados.
    
</div>

### 1.2.2. Distribuição

#### 1.2.2.1. Train / Test

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

pie = ax.pie([len(df1_train), len(df1_test)],
             labels   = ["Train dataset", "Test dataset"],
             colors   = ["salmon", "teal"],
             textprops= {"fontsize": 15},
             autopct  = '%1.1f%%')

ax.axis("equal")
ax.set_title("Comparação de comprimento do conjunto de dados \n", fontsize=18)
fig.set_facecolor('white')
plt.show();

#### 1.2.2.2. Proporção das variáveis

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

plt.pie([len(feature_cat), len(feature_float)], 
        labels=['Categorical', 'Continuos'],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')

#ax.axis("equal")
ax.set_title("Comparação variáveis continuas/categóricas \n Dataset Treino/Teste", fontsize=18)
fig.set_facecolor('white')
plt.show()

#### 1.2.2.1. Target
Vamos ver as ocorrências de números individuais do conjunto de dados de treino.

In [ ]:
%%time 

col = [(1,51), (51,100),  ]
       
for x in col:
       
    L    = len(df1_train[feature_float].columns[x[0]:x[1]])
    nrow = int(np.ceil(L/6))
    ncol = 6
    i    = 1

    remove_last = (nrow * ncol) - L
    fig, ax     = plt.subplots(nrow, ncol,figsize=(24, 30))
    
    fig.subplots_adjust(top=0.95)
    
    for feature in df1_train[feature_float].columns[x[0]:x[1]]:

        plt.subplot(nrow, ncol, i)

        ax = sns.kdeplot(df1_train[feature], shade=True, color='salmon',  alpha=0.5, label='train')
        ax = sns.kdeplot(df1_test[feature], shade=True, color='teal',  alpha=0.5, label='test')
        plt.xlabel(feature, fontsize=9)
        plt.legend()

        i += 1
        
        gc.collect()
    
    plt.suptitle('DistPlot: train & test data de {} à {}'.format(x[0],x[1]), fontsize=20)
    plt.show()
    
    gc.collect()
    
gc.collect()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>

- Os conjuntos de treinamento e teste têm aproximadamente as mesma distribuição em termos de variáveis; <br>
- Temos poucas variáveis com distribuição normal; <br>
- A maioria das variáveis tem distribuições distorcidas. <br>

</div>

#### 1.2.2.1. Detecção de Outlier

##### 1.2.2.1.1. Data Train 

In [ ]:
col = [(1,21), (21,41), (41,61), (61,81), (81,101)]

df_plot = ((df1_train[feature_float] - df1_train[feature_float].min())/
           (df1_train[feature_float].max() - df1_train[feature_float].min()))

fig, ax = plt.subplots(len(col), 1, figsize=(25,30))

for i, (x) in enumerate(col): 
    sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]);
    gc.collect()

##### 1.2.2.1.2. Data Test

In [ ]:
col = [(1,21), (21,41), (41,61), (61,81), (81,101)]

df_plot = ((df1_test[feature_float] - df1_test[feature_float].min())/
           (df1_test[feature_float].max() - df1_test[feature_float].min()))

fig, ax = plt.subplots(len(col), 1, figsize=(25,30))

for i, (x) in enumerate(col): 
    sns.boxplot(data = df_plot.iloc[:, x[0]:x[1] ], ax = ax[i]);
    gc.collect()    
    

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Acima observamos que temos muitos outliers em ambos conjunto de dados, na etapa de processamento vamos fazer o tramento ou remoção dos outliers para ajudar na previsão dos modelos. 

</div>

#### 1.2.2.3. Target
A variável alvo tem os valores 0 e 1, vamos verificar a distribuição da variável `target` que é o nosso alvo de previsão.

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.countplot(x=df1_train['target'], palette='viridis')
ax.set_title('Distribuição da variável Target', fontsize=20, y=1.05)

sns.despine(right=True)
sns.despine(offset=10, trim=True)

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Podemos observar no gráfico acima que não temos desbalanceamento nos dados, temos mais dados na classe possitiva. 
    
</div>

#### 1.2.2.4. Variáveis preditoras  vs Target.

In [ ]:
%%time 

col = [(1,21), (21,41), (41,61), (61,81), (81,100) ]
       
for x in col:
       
    L    = len(df1_train[feature_float].columns[x[0]:x[1]])
    nrow = int(np.ceil(L/6))
    ncol = 6
    i    = 1

    remove_last = (nrow * ncol) - L
    fig, ax     = plt.subplots(nrow, ncol,figsize=(30, 20))
    
    fig.subplots_adjust(top=0.95)
    
    for feature in df1_train[feature_float].columns[x[0]:x[1]]:

        plt.subplot(nrow, ncol, i)

        ax = sns.kdeplot(df1_train[feature], 
                     shade    = True, 
                     palette  = 'viridis',  
                     alpha    = 0.5, 
                     hue      = df1_train['target'], 
                     multiple = "stack")
        
        plt.xlabel(feature, fontsize=9)
       
        i += 1
        
        gc.collect()
    
    plt.suptitle('DistPlot: Variável de treino vs target {} à {}'.format(x[0],x[1]), fontsize=20)
    
    plt.show()
       
    gc.collect()
    
gc.collect()

# <div class="alert alert-success">  2. Split Train/Test </div> <br>

Antes de fazer a divisão dos dados dataset, vamos fazer a redução dos mesmo com a utlização de uma função que modifica os tipos de variáveis dos dataset, ao fazermos isso ganhamos espaço.  

In [ ]:
%%time
df1_train = reduce_memory_usage(df1_train)
df1_test  = reduce_memory_usage(df1_test)

In [ ]:
#jb.dump(df1_train, path + 'pkl/df1_nb_01_train.pkl.z')
#jb.dump(df1_test,  path + 'pkl/df1_nb_01_test.pkl.z')

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Tivemos uma redução em ambos datasets de 74.9%, com isso o dataset de treino passou de 466.9 MB para 117.3 Mb e no dataset de teste passou de 416.1 MB para 105.06 Mb.
    
</div>

In [ ]:
gc.collect()
X      = df1_train.drop(['target', 'id'], axis=1)
y      = df1_train['target']
X_test = df1_test.drop(['id'], axis=1)
cols   = X_test.columns

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 12359)

del df1_train,df1_test

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape , X_test.shape

In [ ]:
X_test.head()

# <div class="alert alert-success">  2. Modelo Baseline XGB </div> <br>

Nesta etapa do processo, vamos utilizar o **XGBClassifier** como linha de base, em relação ao tratamento dos dados vamos fazer apenas o scaler neste momento.

In [ ]:
seed   = 12359
params = {'objective'     : 'binary:logistic',    
          'eval_metric'   : 'auc',
          'random_state'  : seed}

if torch.cuda.is_available():           
    params.update({'predictor'  : 'gpu_predictor', 
                   'tree_method': 'gpu_hist', 
                   'gpu_id'     :  0})

params

In [ ]:
%%time 

model_baseline = xgb.XGBClassifier(**params)

scalers = [None, 
           StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)]

for scaler in scalers: 
    
    if scaler!=None:
        X_train_s = scaler.fit_transform(X_train)
        X_valid_s = scaler.fit_transform(X_valid)
    else:
        X_train_s = X_train
        X_valid_s = X_valid
                
    model_baseline.fit(X_train_s, y_train, verbose = False)
    y_hat = model_baseline.predict_proba(X_valid_s)[:, 1]    
    auc   = metrics.roc_auc_score(y_valid, y_hat) 
    
    print('Validaçao AUC: {:2.5f} => {}'.format(auc, scaler))

    gc.collect()

print()   

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
    
Com scaler QuantileTransformer obtivemos uma AUC de 0.73057, como estamos fazer apenas uma validação simples, neste caso a pontuação do score pode ser afetada por aleatoriedade dos dados, sendo assim, vamos fazer uma validação cruzada para termos uma estimativa robusta.  <br>

Para o treinamento do modelo foi criado uma função,que tem a finalidade de treinar um conjunto de scalers para um determinado modelo, durante o treinamento serão exibidos os resultados e no final será retornado o melhor modelo.
    
</div>

In [ ]:
path='Data/'
def cross_val_model(model, X_train_, y_train_, X_test_,  scalers, name_model, 
                    FOLDS=5, verbose=False, seed=12359, use_ntree_limit=False): 
    
    mdl_train   = []
    feature_imp = 0 
    auc_best    = 0
    
    for scaler in scalers: 
        
        gc.collect()

        df_submission.claim = 0           
        feature_imp_best    = 0       
        auc                 = []
        lloss               = []
        f1                  = []
        ntree               = []
        n_estimators        = model.get_params()['n_estimators'] 
        kfold               = KFold(n_splits=FOLDS, random_state=seed, shuffle=True)

        if scaler!=None:
            X_ts = scaler.fit_transform(X_test_.copy())
        else:
            X_ts = X_test_.copy()

        print('='*80)
        print('Scaler: {} - n_estimators: {}'.format(scaler,n_estimators))
        print('='*80)

        for i, (train_idx, test_idx) in enumerate(kfold.split(X_train_)):

            i+=1

            X_tr, y_tr = X_train_.iloc[train_idx], y_train_.iloc[train_idx]
            X_vl, y_vl = X_train_.iloc[test_idx], y_train_.iloc[test_idx]

            # Scaler
            if scaler!=None:    
                X_tr = scaler.fit_transform(X_tr)
                X_vl = scaler.fit_transform(X_vl)                

            model.fit(X_tr, y_tr, 
                      eval_set              = [(X_tr,y_tr), (X_vl,y_vl)],
                      early_stopping_rounds = int(n_estimators*.1), 
                      verbose               = verbose
                     )
            
            if use_ntree_limit:
                y_hat_prob  = model.predict_proba(X_vl, ntree_limit=model.best_ntree_limit)[:, 1] # 
                best_ntree_ = model.best_ntree_limit
            else: 
                y_hat_prob  = model.predict_proba(X_vl)[:, 1] # 
                best_ntree_ = n_estimators
                            
            y_hat         = (y_hat_prob >.5).astype(int) 
            log_loss_     = metrics.log_loss(y_vl, y_hat_prob)                
            f1_score_     = metrics.f1_score(y_vl, y_hat)                    
            auc_          = metrics.roc_auc_score(y_vl, y_hat_prob)

            stop = '*' if n_estimators > best_ntree_ else ' '
            msg  = '[Fold {}] AUC: {:.5f} - F1: {:.5f} - L. LOSS: {:.5f} {} {}'
            print(msg.format(i, auc_, f1_score_,log_loss_, stop, best_ntree_))

            # Getting mean feature importances (i.e. devided by number of splits)
            feature_imp  += model.feature_importances_ / FOLDS
            
            df_submission['target'] += model.predict_proba(X_ts)[:, 1] / FOLDS

            f1.append(f1_score_)
            lloss.append(log_loss_)
            auc.append(auc_)
            ntree.append(best_ntree_)
            
            gc.collect()
                        
        auc_mean   = np.mean(auc)
        auc_std    = np.std(auc)
        lloss_mean = np.mean(lloss)
        f1_mean    = np.mean(f1)
        ntree_mean = np.mean(ntree)
        
        if auc_mean > auc_best: 
            auc_best          = auc_mean
            f1_best           = f1_mean
            lloss_best        = lloss_mean
            model_best        = model
            feature_imp_best  = feature_imp
            scaler_best       = scaler
                                    
        print('-'*80)
        msg = '[Mean Fold] AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. LOSS: {:.5f} - {} '
        print(msg.format(auc_mean,auc_std, f1_mean, lloss_mean, ntree_mean))
        print('='*80)
        print('')

        # Gerar o arquivo de submissão 
        name_file_sub = 'submission/' + name_model + '_' + str(scaler).lower()[:4] + '.csv'
        df_submission.to_csv(path + name_file_sub.format(auc_mean), index = False)

        gc.collect()
     
    mdl_name_best = 'model/' + name_model.format(auc_mean)
    
    jb.dump(model_best, mdl_name_best)
    
    print()
    print('='*80)
    print('Scaler Best: {}'.format(scaler_best))
    print('AUC        : {:2.5f}'.format(auc_best))
    print('F1-Score   : {:2.5f}'.format(f1_best))
    print('L. Loss    : {:2.5f}'.format(lloss_best))
    print('='*80)
    print()
            
    gc.collect()  
    
    return model_best

In [ ]:
%%time

gc.collect()

params.update({'n_estimators': 100})

scalers = [None, 
           StandardScaler(), 
           RobustScaler(), 
           MinMaxScaler(), 
           MaxAbsScaler(), 
           QuantileTransformer(output_distribution='normal', random_state=0)]

model_best = cross_val_model(model                = xgb.XGBClassifier(**params), 
                             X_train_             = X, 
                             y_train_             = y,
                             X_test_              = X_test,                                            
                             scalers              = scalers, 
                             name_model           = 'xgb_001_bl_{:2.5f}', 
                             FOLDS                = 5, 
                             seed                 = seed, 
                             use_ntree_limit      = False
                             ) 

gc.collect()
print(params)

<div class="alert alert-info" role="alert">

**`NOTA:`** <br>
    
Na validação cruzada obtivemos uma `AUC` de `0.73104` com um desvio padrão de `0.00113` sem fazer o scaler, na submissão obtivemos os seguintes resultados: <br>

- None: 0.73562    
- StandarScaler: 0.73706    
- QuartileTransforme: `0.73780`
    

<br>
    
Com a submissão dos arquivo na competição, observamos que QuartileTransforme resulta no melhor, **agora temos uma baseline que AUC 0.73780**.
    
 
Vamos treinar novamente, sendo que vamos acrescentar o parametros  `n_estimators` que indica o número de arvores para o treinamento do modelo.
    
</div>

In [ ]:
%%time

gc.collect()

params.update({'n_estimators': 1000})

scalers = [None, 
           StandardScaler(), 
           RobustScaler(),           
           QuantileTransformer(output_distribution='normal', random_state=0)]

model_best = cross_val_model(model           = xgb.XGBClassifier(**params), 
                             X_train_        = X, 
                             y_train_        = y,
                             X_test_         = X_test,                                            
                             scalers         = scalers, 
                             name_model      = 'xgb_002_bl_n_estimators_1000_{:2.5f}', 
                             FOLDS           = 5, 
                             seed            = seed, 
                             use_ntree_limit = False
                             ) 

gc.collect()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
No treinamento do modelo, podemos observar uma melhar na AUC de `0.73104`  sem fazer o scaler para `0.73136`, no resultado das submissões dos arquivos obtivemos os seguintes resultados: 


- None: 0.73794
- QuantileTransforme:  `0.73812`
    
    
Mais uma vez o scaler QuantileTransforme se mostrou melhor na submissão e bateu a baseline de `0.73136` para `0.73812`, como podemos observar o aumento de estimadores (arvores) melhorou o score, será que o número de 1000 arvores para o XBG é o ideal? O XGB tem um atributo chamado `best_ntree_limit` que retorna o número aproximado de arvores, que pode ser utilizado na previsão do modelo através do parametro `ntree_limit` do método `predict`ou pridict_proba.  
    
</div>

In [ ]:
%%time

gc.collect()

params.update({'n_estimators': 1000})

scalers = [None, 
           StandardScaler(), 
           RobustScaler(),           
           QuantileTransformer(output_distribution='normal', random_state=0)]

model_best = cross_val_model(model           = xgb.XGBClassifier(**params), 
                             X_train_        = X, 
                             y_train_        = y,
                             X_test_         = X_test,                                            
                             scalers         = scalers, 
                             name_model      = 'xgb_003_bl_n_estimators_1000_limit_{:2.5f}', 
                             FOLDS           = 5, 
                             seed            = seed, 
                             use_ntree_limit = True
                             ) 

gc.collect()

print(params)
print()

<div class="alert alert-info" role="alert"> 
 
**`NOTA:`** <br>
  

Como podemos observar os resultados não melhou na validação, porém ao fazer as submissões dos dois melhores resultados obtivemos os seguites resultados com uma pequena melhoria em ambos os scalres: 
  
- None: 0.73816 
- QuantileTransforme: `0.73818`
  
<br> 
    
O QuantileTransforme novamente se mostrou o melhor scaler para o XGB e outro ponto importante é o número de estimadores que fica em média de 100 à 120 estimadores, vamos fazer uma pequena análise do modelo com esses parametros e scaler quantileTransforme. 
    
</div>

## 2.1. Análise do Modelo 
Vamos fazer o treinamento novamente do modelo com 1000 estimadores e fazer a previsão utilizando o limite ideal sugerido pelo atributo `best_ntree_limit` do XGB, sendo que vamos treinar em 80% dos dados e fazer a previsão em 20% dos dados que o medelo não viu, para termos uma idéia da rubustez do modelo em dados não visto no treinamento.  

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 12359)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape , X_test.shape

In [ ]:
%%time

params.update({'n_estimators': 1000})

scaler     = QuantileTransformer(output_distribution='normal', random_state=0) 
model_best = cross_val_model(model           = xgb.XGBClassifier(**params), 
                             X_train_        = X_train, 
                             y_train_        = y_train,
                             X_test_         = X_test,                                            
                             scalers         = [scaler], 
                             name_model      = 'xgb_004_bl_n_estimators_1000_{:2.5f}', 
                             FOLDS           = 5, 
                             seed            = seed, 
                             use_ntree_limit = False
                             ) 

gc.collect()

print(params)

### 2.1.1. Número de Estimadores

In [ ]:
model_best.get_params()

In [ ]:
results     = model_best.evals_result()
ntree_limit = model_best.best_ntree_limit

plt.figure(figsize=(7,5))
plt.plot(results["validation_0"]["auc"], label="Treinamento")
plt.plot(results["validation_1"]["auc"], label="Validação")


plt.axvline(ntree_limit, 
            color="gray", 
            label="N. de árvore ideal {}".format(ntree_limit))

plt.xlabel("Número de árvores")
plt.ylabel("AUC")
plt.legend();

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
Acima recuperamos as informações de treinamento do nosso modelo, podemos observar que o número de 1000 estimadores é mais que suficiente para o treinamento do modelo, o ideal é que fique em entre de 110 à 120 para esses dados e com a utilização dos parametros padrões que devem ser ajustados para o `XGB`. <br>
    
    
Vamos agora utilizar o modelo treinado que foi retornado pela função e vamos fazer a previsão para novos dados que o modelo não viu no treinamento, para termos uma ideia da generalização do modelo, lembrando que o modelo que foi treinado utiliza 1000 estimadores (arvores), sendo assim, vamos utilizar na previssão 114 estimadores utilizando o parametro `ntree_limit` ao fazermos as previsões.

</div>

In [ ]:
X_valid_sc  = scaler.fit_transform(X_valid)

In [ ]:
%%time

threshold   =.5
y_pred_prob = model_best.predict_proba(X_valid_sc,  ntree_limit=ntree_limit)[:, 1] 
y_pred      = (y_pred_prob > threshold).astype(int)

f1_    = metrics.f1_score(y_valid, y_pred)
auc_   = metrics.roc_auc_score(y_valid, y_pred_prob)
lloss_ = metrics.log_loss(y_valid, y_pred_prob) 
    
print('AUC     : {:2.5f}'.format(auc_))
print('F1-Score: {:2.5f}'.format(f1_))
print('L. Loss : {:2.5f}'.format(lloss_))
print()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>

No treinamento obtivemos uma AUC de 0.72913 e no dados de validação obtivemos uma AUC de 0.72866, temos uma pequena diferença, o que nos indica que o modelo consegue generalizar em dados não visto no treinamento.  
    
</div>

### 2.1.2. Curva ROC

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_pred_prob)

plot_roc_curve(fpr, tpr, label="XGB")
plt.show()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
    
Observando a Curva ROC acima, podemos fazer alguns testes em relação ao ponto de corte, pois a AUC de 0.72866 tem um ponto de corte de 0.5 que é padrão, o que nos dar um F1-score de 0.69587, vamos fazer um gráfico de Matriz de Confusão para termos uma ideia melhor das previsões do modelo.  
    
</div>

### 2.1.3. Matriz de Confusão

#### 2.1.3.1. Com ponto de corte padrão

In [ ]:
threshold    = .5
y_pred_valid = (y_pred_prob > threshold).astype(int)
f1_          = metrics.f1_score (y_valid, y_pred_valid)
auc_         = metrics.roc_auc_score(y_valid, y_pred_prob)

print(metrics.classification_report(y_valid, y_pred_valid))
print('')
print('AUC     : {:2.5f}'.format(auc_))
print('F1-score: {:2.5f}'.format(f1_))

confusion_plot(metrics.confusion_matrix(y_valid, y_pred) )

#### 2.1.3.2. Com ponto de corte

In [ ]:
threshold    = .43
y_pred_valid = (y_pred_prob > threshold).astype(int)
f1_          = metrics.f1_score (y_valid, y_pred_valid)
auc_         = metrics.roc_auc_score(y_valid, y_pred_prob)

print(metrics.classification_report(y_valid, y_pred_valid))
print('')
print('AUC     : {:2.5f}'.format(auc_))
print('F1-score: {:2.5f}'.format(f1_))

confusion_plot(metrics.confusion_matrix(y_valid, y_pred_valid) )

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
Com um ponto de corte de 0.43 temos um F1-score de 0.71225, passamos a acerta mais os falso positivos. 
    
</div>


### 2.1.3. Feature Importances  

In [ ]:
feature_imp_     = model_best.feature_importances_

df               = pd.DataFrame()
df["Feature"]    = X.columns
df["Importance"] = feature_imp_ / feature_imp_.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 70))
bars    = ax.barh(df["Feature"], 
                  df["Importance"], 
                  height    = 0.4,
                  color     = "mediumorchid", 
                  edgecolor = "black")

ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
#ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adicionando rótulos na parte superior
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverter a direção do eixo y 
plt.gca().invert_yaxis()

# <div class="alert alert-success">  3. Conclusão </div> <br>

<div class="alert alert-info" role="alert"> 

Com este notebook fizemos uma pequena análise dos dados e identificamos que os dados disponíveis para a competição não segue uma distribuição normal, tem muitos outliers que precisam de um tratamento, nesta etapa utilizamos o XGB como modelo de linha de base e testamos diversos tipos de padronização de dados, o `QuatileTransforme` se mostrou o melhor padronizador com o XGB com a maioria dos parametros padrão.   

<br>     
No próximo notebook vamos criar novas variáveis para ajuda os modelos a identificar novos padrões nos dados e melhor AUC.  
    
    
</div>
